In [2]:
import sys
import numpy as np
from glob import glob
from keras.applications.vgg16 import VGG16, preprocess_input, decode_predictions
from keras.models import load_model
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model
from keras.layers import Input, Activation, Dropout, Flatten, Dense
from keras import optimizers
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [3]:
img_rows, img_cols = 224,224
input_tensor = Input(shape=(img_rows, img_cols, 3))
vgg16 = VGG16(include_top=False, weights='imagenet', input_tensor=input_tensor)

In [4]:
# filename = './1_datasets/train/cat.0.jpg'

# # 引数で指定した画像ファイルを読み込む
# # サイズはVGG16のデフォルトである224x224にリサイズされる
# img = image.load_img(filename, target_size=(224, 224))

# # 読み込んだPIL形式の画像をarrayに変換
# x = image.img_to_array(img)
# print(x.shape)

# # 3次元テンソル（rows, cols, channels) を
# # 4次元テンソル (samples, rows, cols, channels) に変換
# # 入力画像は1枚なのでsamples=1でよい
# x = np.expand_dims(x, axis=0)

# # Top-5のクラスを予測する
# # VGG16の1000クラスはdecode_predictions()で文字列に変換される
# preds = vgg16.predict(preprocess_input(x))
# results = decode_predictions(preds, top=5)[0]
# for result in results:
#     print(result)

In [5]:
# im = image.open(filename, 'r')
# plt.imshow(img)

In [6]:
print(vgg16.output)
print(vgg16.output_shape[1])

Tensor("block5_pool/MaxPool:0", shape=(?, 7, 7, 512), dtype=float32)
7


In [7]:
nb_classes = ['cat', 'dog']
fc_model = Sequential()
fc_model.add(Flatten(input_shape=vgg16.output_shape[1:]))
fc_model.add(Dense(256, activation='relu'))
fc_model.add(Dropout(0.5))
fc_model.add(Dense(len(nb_classes), activation='softmax'))
model = Model(input=vgg16.input, output=fc_model(vgg16.output))

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:7: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("se..., inputs=Tensor("in...)`
  import sys


In [8]:
for i, layer in enumerate(model.layers):
   print(i, layer.name)

0 input_1
1 block1_conv1
2 block1_conv2
3 block1_pool
4 block2_conv1
5 block2_conv2
6 block2_pool
7 block3_conv1
8 block3_conv2
9 block3_conv3
10 block3_pool
11 block4_conv1
12 block4_conv2
13 block4_conv3
14 block4_pool
15 block5_conv1
16 block5_conv2
17 block5_conv3
18 block5_pool
19 sequential_1


In [9]:
for layer in model.layers[:18]:
    layer.trainable = False
    
# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer=optimizers.SGD(lr=1e-4, momentum=0.9), loss='categorical_crossentropy',metrics=['accuracy'])

In [10]:
for i, layer in enumerate(model.layers):
   print(i, layer.name)

0 input_1
1 block1_conv1
2 block1_conv2
3 block1_pool
4 block2_conv1
5 block2_conv2
6 block2_pool
7 block3_conv1
8 block3_conv2
9 block3_conv3
10 block3_pool
11 block4_conv1
12 block4_conv2
13 block4_conv3
14 block4_pool
15 block5_conv1
16 block5_conv2
17 block5_conv3
18 block5_pool
19 sequential_1


In [ ]:
train_data_dir = '1_datasets/train'
validation_data_dir = '1_datasets/validation'
img_width, img_height = 224,224
batch_size = 16
nb_train_samples = 200
nb_validation_samples = 50
epochs = 4
train_datagen = ImageDataGenerator(
    rescale = 1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    classes=nb_classes)

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    classes=nb_classes)

Found 20000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.


In [ ]:
history = model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size)

model.save_weights('finetuning.h5')
# save_history(history, os.path.join(result_dir, 'history_finetuning.txt'))

Epoch 1/4
